In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
tqdm.pandas()
from openai import OpenAI
import os

from langchain_openai import OpenAIEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

pd.set_option('display.max_columns',None)

<h3> 1. Importing ABT</h3>

In [31]:
df_feat = pd.read_parquet('../data/02_pre_proc/ABT_feats.parquet')
df_feat.sample()

,USER_ID,FK_GAME_NAME,GAMEPLAY_HOURS,release_from_now,Estimated owners,Peak CCU,Price,DLC count,About the game,Supported languages,Reviews,Achievements,Recommendations,Notes,Categories,Genres,Tags
3286,101487265,freestyle2streetbasketball,15.8,3616.0,7500000.0,391,0.0,15,Freestyle 2 is the Ultimate Online Street Bask...,"['English', 'German', 'French', 'Thai', 'Japan...",“Freestyle 2 is an incredibly polished basketb...,0,150,None,"Multi-player,MMO,PvP,Online PvP,Co-op,Online C...","Action,Casual,Free to Play,Massively Multiplay...","Free to Play,Basketball,Sports,Multiplayer,Mas..."


In [32]:
print(df_feat.FK_GAME_NAME.nunique())
df_content = df_feat.drop_duplicates(subset=['FK_GAME_NAME'])
df_train = df_content.sample(2000, random_state=1)
df_validation = df_content[~df_content.FK_GAME_NAME.isin(df_train.FK_GAME_NAME)]

3455


In [ ]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=os.getenv('openai_apikey')
    )

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

doc_list=[]
index_list=[]
for i in df_train.iterrows():
    about = i[1]['About the game']
    category = i[1]['Categories']
    tags = i[1]['Tags']
    ID = i[1]['FK_GAME_NAME']


    doc = Document(
        page_content=str(about),
        metadata={
            "category": str(category),
            "tags": str(tags)
            }
        )
    doc_list.append(doc)
    index_list.append(ID)
    
vector_store.add_documents(
  documents = doc_list,
  ids=index_list
  )

In [ ]:
teste = df_feat[
    df_feat.USER_ID==df_train.USER_ID.sample().item()
]
game_teste = teste.FK_GAME_NAME.unique().sample()

about = teste[teste.FK_GAME_NAME==game_teste]['About the game'].item()
category = teste[teste.FK_GAME_NAME==game_teste]['Categories'].item()
tags = teste[teste.FK_GAME_NAME==game_teste]['Tags'].item()

print('>>Inference:','\nGame:',game_teste, '\nAbout:',about,'\ncategory',category,'\ntags',tags)
teste.FK_GAME_NAME.unique()

In [ ]:
results = vector_store.similarity_search(
    str(about),
    k=6
    )

[i.id for i  in results[1:]]

In [ ]:
vector_store = FAISS(
            embedding_function=client.embeddings,
            index=,
            docstore=InMemoryDocstore(),
            index_to_docstore_id={})

AttributeError: 'Embeddings' object has no attribute 'embed_query'

In [ ]:
vector_store = FAISS(
            embedding_function=embeddings,
            index=faiss.IndexFlatL2(len(embeddings.embed_query("hello world"))),
            docstore=InMemoryDocstore(),
            index_to_docstore_id={})

vector_store.add_documents(
    documents=[Document(i) for i in dict_conteudo[LU].values()],
    ids=list(dict_conteudo[LU].keys()))

results = vector_store.similarity_search(questao_texto, k=2)

In [ ]:
client = OpenAI(api_key = os.getenv('openai_apikey'))
model="text-embedding-3-small"
 
def generate_embeddings(text, client, model):
    text=[text]
    response = client.embeddings.create(
        input = text,
        model=model).data[0].embedding
    embeddings = response['data'][0]['embedding']
    return embeddings

embedding_feats=['About the game','Supported languages','Reviews','Categories','Genres','Tags']
for col in embedding_feats:
    print(col)
    df_feat[col] = df_feat[col].fillna(f'Sem dados de {col}')
    df_feat[col] = df_feat[col].astype(str)
    df_feat[col] = df_feat[col].progress_apply(lambda x: generate_embeddings(x, client, model))